# НАЧАЛО

## Импорт библиотек

In [4]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta

## Данные

In [5]:
DATA_PATH = "../data/"
SAVE_PATH = "../charts/"

# Загружаем данные
BETA = pd.read_excel(DATA_PATH + "beta_gibbs.xlsx", header=None).values

X_MAT = pd.read_excel(DATA_PATH + "X.xlsx", header=None).values

SHOCKS_AS          = pd.read_excel(DATA_PATH + "ETA_record AS.xlsx",        header=None).values
SHOCKS_PRIVATED_AD = pd.read_excel(DATA_PATH + "ETA_record PrivateAD.xlsx", header=None).values

SHOCKS_MP = pd.read_excel(DATA_PATH + "ETA_record MP.xlsx", header=None).values
SHOCKS_FP = pd.read_excel(DATA_PATH + "ETA_record FP.xlsx", header=None).values

A0INV_P = pd.read_excel(DATA_PATH + "struct_irf_record for p.xlsx", header=None).values
A0INV_Y = pd.read_excel(DATA_PATH + "struct_irf_record for Y.xlsx", header=None).values
A0INV_I = pd.read_excel(DATA_PATH + "struct_irf_record for i.xlsx", header=None).values
A0INV_B = pd.read_excel(DATA_PATH + "struct_irf_record for B.xlsx", header=None).values

ACTUAL_P = pd.read_excel(DATA_PATH + 'actual_p.xlsx', header=None)[0].tolist()
ACTUAL_I = pd.read_excel(DATA_PATH + 'actual_i.xlsx', header=None)[0].tolist()

P_OIL_FORECAST = pd.read_excel(DATA_PATH + "poil_forecast.xlsx", header=None)[0].tolist()

## Константы

### Параметры модели

In [6]:
T = X_MAT.shape[0]
N_ITER = BETA.shape[1]
N_VARS = 4
N_LAGS = 2
N_REG = 10

In [7]:
# Индексы переменных
P = 0
Y = 1
I = 2
B = 3
I_TARGET = 7.5

# I. КОНТРФАКТИЧЕСКИЙ АНАЛИЗ
T_FIX = 73
LAST_POINT = T - T_FIX + 1
QUARTERS = [datetime(2005, 7, 1) + timedelta(days=91.25 * i) for i in range(T)]

# II. УСЛОВНЫЙ ПРОГНОЗ
H = 8  # 2 года (8 кварталов)
I_TARGET_CONDITIONAL = 21
T_FORECAST = T + H  # 78 + 8 = 86 периодов
LAST_POINT_FORECAST = LAST_POINT + 3
QUARTERS_FORECAST = [datetime(2005, 7, 1) + timedelta(days=91.25 * i) for i in range(T_FORECAST)]

## Функции

In [8]:
def unpack_parameters(beta_i: np.ndarray) -> tuple[np.ndarray, ...]:
    """Распаковываем коэффициенты VAR из вектора параметров
    
    Args:
        beta_i (np.ndarray): _description_

    Returns:
        tuple[np.ndarray, ...]: _description_
    """
    return beta_i[0:10], beta_i[10:20], beta_i[20:30], beta_i[30:40]


def get_initial_values(T_FIX: int, T: int, X_MAT: np.ndarray, B_p: np.ndarray, B_y: np.ndarray, B_i: np.ndarray, B_b: np.ndarray, A0inv_full: np.ndarray,
                       shocks_as: np.ndarray, shocks_ad: np.ndarray, shocks_mp: np.ndarray, shocks_fp: np.ndarray) -> np.ndarray:
    """Восстанавливаем начальные значения переменных до T_FIX
    
    Args:
        T_FIX (_tintype_): _description_
        T (int): _description_
        X_MAT (np.ndarray): _description_
        B_p (np.ndarray): _description_
        B_y (np.ndarray): _description_
        B_i (np.ndarray): _description_
        B_b (np.ndarray): _description_
        A0inv_full (np.ndarray): _description_
        shocks_as (np.ndarray): Шоки
        shocks_ad (np.ndarray): Шоки
        shocks_mp (np.ndarray): Шоки
        shocks_fp (np.ndarray): Шоки

    Returns:
        np.ndarray: _description_
    """
    Y_temp = np.zeros((T, N_VARS))  
    
    for t in range(T_FIX):
        x_t = X_MAT[t, :]
        eps_t = np.vstack([shocks_as[t], shocks_ad[t], shocks_mp[t], shocks_fp[t]])
        
        Y_temp[t, P] = (B_p.T @ x_t + A0inv_full[P, :] @ eps_t).item()
        Y_temp[t, Y] = (B_y.T @ x_t + A0inv_full[Y, :] @ eps_t).item()
        Y_temp[t, I] = (B_i.T @ x_t + A0inv_full[I, :] @ eps_t).item()
        Y_temp[t, B] = (B_b.T @ x_t + A0inv_full[B, :] @ eps_t).item()
    
    return Y_temp


def analyze_and_plot(CF: np.ndarray, title: str, name: str, file_name: str, quarters: list, actual, last_point: int, yaxis_title: str="Инфляция (%)") -> tuple[np.ndarray, ...]:
    """Рисуем и сохраняем интерактивный go график.

    Args:
        CF (np.ndarray): _description_
        title (str): Имя графика
        name (str): Имя ряда
        file_name (str): Имя файла, для сохранения
        quarters (list): Временная квартальная шкала
        last_point (int): _description_
        yaxis_title (str): Имя оси Y

    Returns:
        tuple[np.ndarray, ...]: _description_
    """

    median = np.nanmedian(CF, axis=1)
    low = np.nanpercentile(CF, 16, axis=1)
    high = np.nanpercentile(CF, 84, axis=1)

    fig = go.Figure()

    # Добавляем линии на график
    fig.add_trace(go.Scatter(x=quarters[-last_point:], y=median[-last_point:], mode='lines', name=name, line=dict(color="#0000ff", width=3)))
    
    fig.add_trace(go.Scatter(x=quarters[-last_point:], y=low[-last_point:], mode='lines',
                             name='68% ДИ: нижняя', line=dict(color="#0000ff", width=1, dash='dash')))

    fig.add_trace(go.Scatter(x=quarters[-last_point:], y=high[-last_point:], mode='lines',
                             name='68% ДИ: верхняя', line=dict(color="#0000ff", width=1, dash='dash')))

    fig.add_trace(go.Scatter(x=quarters, y=actual, mode='lines',
                             name='Фактическая', line=dict(color="#000000", width=3)))
    
    # Настраиваем оформление
    fig.update_layout(title=title, xaxis_title='Квартал', yaxis_title=yaxis_title, plot_bgcolor="#ffffff", showlegend=True,
                      paper_bgcolor="#ffffff", legend=dict(bordercolor="#000000", borderwidth=1),
                      shapes=[dict(type="rect", xref="paper", yref="paper", x0=0, y0=0, x1=1, y1=1, line=dict(color="#000000", width=1))])

    # Добавляем сетку
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="#d3d3d3")
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="#d3d3d3")

    fig.show()

    # Сохранить график
    fig.write_html(f"{SAVE_PATH}{file_name}.html")

    return low, median, high

# I. КОНТРФАКТИЧЕСКИЙ АНАЛИЗ

## 1) Фиксированная ключевая ставка (i) = 7,5

In [69]:
# Готовим объект чтобы потом мочь посмотреть новые шоки ДКП
CF_EPS_MP = np.zeros((N_ITER, T))

# Готовим трехмерный объект для хранения 1000 контрфактических итераций по 4 переменным
CF = np.zeros((T, N_ITER, N_VARS))

for i in range(N_ITER):
    # Восстанавливаем коэффициенты VAR
    B_p, B_y, B_i, B_b = unpack_parameters(beta_i=BETA[:, i])
    
    # Структурная матрица A0^{-1} для каждой итерации
    A0inv_full = np.vstack([A0INV_P[i, :], A0INV_Y[i, :], A0INV_I[i, :],  A0INV_B[i, :]])
    
    # Шоки
    shocks_as = SHOCKS_AS[i, :].reshape(-1, 1)
    shocks_ad = SHOCKS_PRIVATED_AD[i, :].reshape(-1, 1)
    shocks_mp = SHOCKS_MP[i, :].reshape(-1, 1)
    shocks_fp = SHOCKS_FP[i, :].reshape(-1, 1)
    
    # Восстанавливаем начальные значения переменных до t_fix
    Y_temp = get_initial_values(T_FIX=T_FIX, T=T, X_MAT=X_MAT, B_p=B_p, B_y=B_y, B_i=B_i, B_b=B_b, A0inv_full=A0inv_full,
                                shocks_as=shocks_as, shocks_ad=shocks_ad, shocks_mp=shocks_mp, shocks_fp=shocks_fp)
    
    # Считаем контрфакт с момента фиксации
    for t in range(T_FIX, T):
        # Собираем лаги (размер (4,))
        l1 = Y_temp[t-1, :].reshape(-1, 1)
        l2 = Y_temp[t-2, :].reshape(-1, 1)

        # Формируем x_sim как вектор [10 x 1]
        x_sim = np.vstack([l1, l2, X_MAT[t, 8], X_MAT[t, 9]])
        
        # Вычисляем шоки
        eps_as = shocks_as[t]
        eps_ad = shocks_ad[t]
        eps_fp = shocks_fp[t]
        
        # Вычисляем eps_mp
        numerator = I_TARGET - (B_i.T @ x_sim + A0inv_full[I, [0, 1, 3]] @ np.vstack([eps_as, eps_ad, eps_fp]))
        eps_mp = (numerator / A0inv_full[I, 2]).item()
        
        # Собираем вектор шоков [4 x 1]
        eps_t = np.vstack([eps_as, eps_ad, eps_mp, eps_fp])
        
        # Обновляем Y_temp
        Y_temp[t, P] = (B_p.T @ x_sim + A0inv_full[P, :] @ eps_t).item()
        Y_temp[t, Y] = (B_y.T @ x_sim + A0inv_full[Y, :] @ eps_t).item()
        Y_temp[t, I] = I_TARGET  # фиксируем ставку
        Y_temp[t, B] = (B_b.T @ x_sim + A0inv_full[B, :] @ eps_t).item()
        
        # Обновляем шок MP
        shocks_mp[t] = eps_mp.item() if isinstance(eps_mp, np.ndarray) else eps_mp
    
    # Сохраняем результат симуляции
    CF[:, i, :] = Y_temp
    CF_EPS_MP[i, :] = shocks_mp.ravel()

p_low, p_median, p_high = analyze_and_plot(CF=CF[:, :, P], title='Инфляция, QoQ: фактическая vs контрфактическая (КС = 7,5% с 3 квартала 2023 г.)',
                                           name='Контрфакт (КС = 7,5%)', file_name='I_КА_1_Фиксированная_ключевая_ставка',
                                           last_point=LAST_POINT, quarters=QUARTERS_FORECAST, actual=ACTUAL_P) 

## 2) Нулевые шоки ДКП

In [70]:
# Готовим объект чтобы потом мочь посмотреть новые шоки ДКП
CF_ZERO_MP = np.zeros((T, N_ITER, N_VARS))

for i in range(N_ITER):
    # Восстанавливаем коэффициенты VAR
    B_p, B_y, B_i, B_b = unpack_parameters(beta_i=BETA[:, i])
    
    # Структурная матрица A0^{-1} для каждой итерации
    A0inv_full = np.vstack([A0INV_P[i, :], A0INV_Y[i, :], A0INV_I[i, :],  A0INV_B[i, :]])
    
    # Шоки
    shocks_as = SHOCKS_AS[i, :]
    shocks_ad = SHOCKS_PRIVATED_AD[i, :]
    shocks_mp = SHOCKS_MP[i, :]
    shocks_fp = SHOCKS_FP[i, :]
    
    # Восстанавливаем начальные значения переменных до t_fix
    Y_temp = get_initial_values(T_FIX=T_FIX, T=T, X_MAT=X_MAT, B_p=B_p, B_y=B_y, B_i=B_i, B_b=B_b, A0inv_full=A0inv_full,
                                shocks_as=shocks_as, shocks_ad=shocks_ad, shocks_mp=shocks_mp, shocks_fp=shocks_fp)

    
    # С 2023Q3 задаём MP-шок = 0
    for t in range(T_FIX, T):
        # Собираем лаги (размер (4,))
        l1 = Y_temp[t-1, :].reshape(-1, 1)
        l2 = Y_temp[t-2, :].reshape(-1, 1)

        # Формируем x_sim как вектор [10 x 1]
        x_sim = np.vstack([l1, l2, X_MAT[t, 8], X_MAT[t, 9]])
        
        eps_zero = np.vstack([shocks_as[t], shocks_ad[t], 0, shocks_fp[t]])
        
        # Обновляем Y_temp
        Y_temp[t, P] = (B_p.T @ x_sim + A0inv_full[P, :] @ eps_zero).item()
        Y_temp[t, Y] = (B_y.T @ x_sim + A0inv_full[Y, :] @ eps_zero).item()
        Y_temp[t, I] = (B_i.T @ x_sim + A0inv_full[I, :] @ eps_zero).item()
        Y_temp[t, B] = (B_b.T @ x_sim + A0inv_full[B, :] @ eps_zero).item()
    
    # Сохраняем результат симуляции
    CF_ZERO_MP[:, i, :] = Y_temp

# Строим графики. Инфляция
p_low, p_median, p_high = analyze_and_plot(CF=CF_ZERO_MP[:, :, P], title='Инфляция, QoQ: фактическая vs контрфактическая (шок ДКП = 0 с 3 квартала 2023 г.)',
                                           name='Контрфакт (шок ДКП = 0)', file_name='I_КА_2_Нулевые_шоки_ДКП_Инфляция',
                                           last_point=LAST_POINT, quarters=QUARTERS_FORECAST, actual=ACTUAL_P)

# Строим графики. Ставка
i_low, i_median, i_high = analyze_and_plot(CF=CF_ZERO_MP[:, :, I], title='Среднеквартальная ключевая ставка: фактическая vs контрфактическая (шок ДКП = 0 с 3 квартала 2023 г.)',
                                           name='Контрфакт (шок ДКП = 0)', file_name='I_КА_2_Нулевые_шоки_ДКП_Ставка',
                                           last_point=LAST_POINT, quarters=QUARTERS, actual=ACTUAL_I, yaxis_title="Ключевая ставка (%)")

# II. УСЛОВНЫЙ ПРОГНОЗ

## 1) Cтавка = 21 с 2025Q1, все шоки = 0, кроме MP

In [ ]:
CF_FORECAST = np.zeros((T_FORECAST, N_ITER, N_VARS))  # [86 x 1000 x 4] - основная матрица прогноза
CF_EPS_MP_FORECAST = np.zeros((N_ITER, T_FORECAST))

# Расширим матрицу Xmat экзогенными переменными на прогнозный период: [86 x 10]
X_MAT_FORECAST = np.zeros((T_FORECAST, N_REG))
X_MAT_FORECAST[:T, :] = X_MAT

for i in range(N_ITER):
    # Восстанавливаем коэффициенты VAR
    B_p, B_y, B_i, B_b = unpack_parameters(beta_i=BETA[:, i])
    
    A0inv_full = np.vstack([A0INV_P[i, :], A0INV_Y[i, :], A0INV_I[i, :],  A0INV_B[i, :]])
    
    shocks_as = SHOCKS_AS[i, :]
    shocks_ad = SHOCKS_PRIVATED_AD[i, :]
    shocks_mp = SHOCKS_MP[i, :]
    shocks_fp = SHOCKS_FP[i, :]

    # 1. Историческая часть (t = 1:78)
    Y_temp = np.zeros((T_FORECAST, N_VARS))  
    
    for t in range(T):
        x_t = X_MAT_FORECAST[t, :]
        eps_t = np.vstack([shocks_as[t], shocks_ad[t], shocks_mp[t], shocks_fp[t]])
        
        Y_temp[t, P] = (B_p.T @ x_t + A0inv_full[P, :] @ eps_t).item()
        Y_temp[t, Y] = (B_y.T @ x_t + A0inv_full[Y, :] @ eps_t).item()
        Y_temp[t, I] = (B_i.T @ x_t + A0inv_full[I, :] @ eps_t).item()
        Y_temp[t, B] = (B_b.T @ x_t + A0inv_full[B, :] @ eps_t).item()

        CF_EPS_MP_FORECAST[i, t] = shocks_mp[t].item()
    
    # 2. Прогнозная часть (t = 79:86)
    for t in range(T, T_FORECAST):
        # Лаги: исторические или из прогноза
        lag1 = Y_temp[t-1, :].reshape(-1, 1)
        lag2 = Y_temp[t-2, :].reshape(-1, 1)

        # Формируем строку регрессоров
        X_MAT_FORECAST[t, :4]  = lag1.flatten()  # первые 4 элемента - lag1
        X_MAT_FORECAST[t, 4:8] = lag2.flatten()  # следующие 4 элемента - lag2
        X_MAT_FORECAST[t, 8]   = 1  # константа
        X_MAT_FORECAST[t, 9]   = P_OIL_FORECAST[t - T]  # нефть из прогноза

        x_t = X_MAT_FORECAST[t, :]

        # Вычисляем MP-шок, который даёт ставку = 21
        eps_mp = (I_TARGET_CONDITIONAL - B_i.T @ x_t) / A0inv_full[I, 2]
        eps_t = np.vstack([0, 0, eps_mp, 0])  # Все шоки, кроме MP, равны 0

        # Пересчитываем переменные
        Y_temp[t, P] = (B_p.T @ x_t + A0inv_full[P, :] @ eps_t).item()
        Y_temp[t, Y] = (B_y.T @ x_t + A0inv_full[Y, :] @ eps_t).item()
        Y_temp[t, I] = I_TARGET_CONDITIONAL
        Y_temp[t, B] = (B_b.T @ x_t + A0inv_full[B, :] @ eps_t).item()

        CF_EPS_MP_FORECAST[i, t] = eps_mp.item()

    CF_FORECAST[:, i, :] = Y_temp


p_low, p_median, p_high = analyze_and_plot(CF=CF_FORECAST[:, :, P], title='Инфляция: факт + прогноз (ставка = 21)',
                                           name='Контрфакт (шок ДКП = 0)', file_name='II_УП_1_Инфляция',
                                           last_point=LAST_POINT_FORECAST, quarters=QUARTERS_FORECAST, actual=ACTUAL_P)

## 2) Ставка = 21 с 2025Q1, шоки спроса

In [ ]:
CF_FORECAST_AD = np.zeros((T_FORECAST, N_ITER, N_VARS))  # [86 x 1000 x 4]
CF_EPS_MP_FORECAST_AD = np.zeros((N_ITER, T_FORECAST))  # [1000 x 86]

X_MAT_FORECAST_AD = np.zeros((T_FORECAST, N_REG))
X_MAT_FORECAST_AD[:T, :] = X_MAT

for i in range(N_ITER):
    # Восстанавливаем коэффициенты VAR
    B_p, B_y, B_i, B_b = unpack_parameters(beta_i=BETA[:, i])

    A0inv_full = np.vstack([A0INV_P[i, :], A0INV_Y[i, :], A0INV_I[i, :],  A0INV_B[i, :]])

    shocks_as = SHOCKS_AS[i, :]
    shocks_ad = SHOCKS_PRIVATED_AD[i, :]
    shocks_mp = SHOCKS_MP[i, :]
    shocks_fp = SHOCKS_FP[i, :]

    # Среднее значение шока AD за 2024 год (t = 75:78)
    ad_mean = np.mean(shocks_ad[74:78])

    # 1. Историческая часть (t = 1:78)
    Y_temp = np.zeros((T_FORECAST, N_VARS))

    for t in range(T):
        x_t = X_MAT_FORECAST_AD[t, :]
        eps_t = np.vstack([shocks_as[t], shocks_ad[t], shocks_mp[t], shocks_fp[t]])

        Y_temp[t, P] = (B_p.T @ x_t + A0inv_full[P, :] @ eps_t).item()
        Y_temp[t, Y] = (B_y.T @ x_t + A0inv_full[Y, :] @ eps_t).item()
        Y_temp[t, I] = (B_i.T @ x_t + A0inv_full[I, :] @ eps_t).item()
        Y_temp[t, B] = (B_b.T @ x_t + A0inv_full[B, :] @ eps_t).item()

        CF_EPS_MP_FORECAST_AD[i, t] = shocks_mp[t].item()

    # 2. Прогнозная часть (t = 79:86)
    for t in range(T, T_FORECAST):
        lag1 = Y_temp[t-1, :].reshape(-1, 1)
        lag2 = Y_temp[t-2, :].reshape(-1, 1)

        X_MAT_FORECAST_AD[t, :4]  = lag1.flatten()  # первые 4 элемента - lag1
        X_MAT_FORECAST_AD[t, 4:8] = lag2.flatten()  # следующие 4 элемента - lag2
        X_MAT_FORECAST_AD[t, 8]   = 1  # константа
        X_MAT_FORECAST_AD[t, 9]   = P_OIL_FORECAST[t - T]  # нефть из прогноза

        x_t = X_MAT_FORECAST_AD[t, :]

        eps_mp = (I_TARGET_CONDITIONAL - B_i.T @ x_t - A0inv_full[I, 1] * ad_mean) / A0inv_full[I, 2]

        eps_t = np.vstack([0, ad_mean, eps_mp, 0])

        Y_temp[t, P] = (B_p.T @ x_t + A0inv_full[P, :] @ eps_t).item()
        Y_temp[t, Y] = (B_y.T @ x_t + A0inv_full[Y, :] @ eps_t).item()
        Y_temp[t, I] = I_TARGET_CONDITIONAL
        Y_temp[t, B] = (B_b.T @ x_t + A0inv_full[B, :] @ eps_t).item()

        CF_EPS_MP_FORECAST_AD[i, t] = eps_mp.item()

    CF_FORECAST_AD[:, i, :] = Y_temp


p_low, p_median, p_high = analyze_and_plot(CF=CF_FORECAST_AD[:, :, P], title='Инфляция: факт + прогноз (ставка = 21)',
                                           name='Контрфакт (шок спроса = 0)', file_name='II_УП_2_Спрос',
                                           last_point=LAST_POINT_FORECAST, quarters=QUARTERS_FORECAST, actual=ACTUAL_P)